In [1]:
Output = c("/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1011. Emission Mixtures/P1011.3. Analyses/P1011.3.3. Biomarker Distribution Analysis/Output")
cur_date = "082223"

library(readxl)
library(openxlsx)
library(tidyverse)
library(reshape2)
library(rlang)
library(PMCMRplus)

# reading in files
cytokine_df = data.frame(read_excel("Input/Processed_Cyotkine_Data_081723.xlsx"))
mRNA_df = data.frame(read_excel("Input/Processed_mRNA_Data_081723.xlsx"))

# making some cols numeric
cytokine_df$Condensate_Conc = as.numeric(cytokine_df$Condensate_Conc)
mRNA_df$Concentration = as.numeric(mRNA_df$Concentration)
mRNA_df$Time_Point = as.numeric(mRNA_df$Time_Point)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.2     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.0
✔ ggplot2   3.4.2     ✔ tibble    3.2.1
✔ lubridate 1.9.2     ✔ tidyr     1.3.0
✔ purrr     1.0.1     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘reshape2’


The following object is masked from ‘package:tidyr’:

    smiths



Attaching package: ‘rlang’


The following objects are masked from ‘package:purrr’:

    %@%, flatten, flatten_chr, flatten_dbl, flatten_int, flatten_lgl,
    flatten_raw, invoke, splice


Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”
Warning message in eval(expr, envir, enclos):
“NAs introduced by coercion”


In [2]:
head(cytokine_df)
head(mRNA_df)

,Subject_ID,Subject_No,Cytokine,Condensate,Burn_Condition,Condensate_Conc,Norm_Cytokine_Conc
,<chr>,<dbl>,<chr>,<chr>,<chr>,<dbl>,<dbl>
1,F_1,1,Eotaxin3,C,F,1,5.438924
2,F_1,1,Eotaxin3,C,F,25,3.630025
3,F_1,1,Eotaxin3,C,F,5,4.694272
4,F_1,1,Eotaxin3,C,S,1,6.165522
5,F_1,1,Eotaxin3,C,S,25,3.703871
6,F_1,1,Eotaxin3,C,S,5,4.562595


,Subject_No,Subject_ID,mRNA,Condensate,Burn_Condition,Concentration,Time_Point,Norm_ddCT
,<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
1,6,M_6,HMOX1,PBS,PBS,NA,24,3.912311
2,5,F_5,HMOX1,PBS,PBS,NA,4,3.067190
3,1,F_1,HMOX1,PBS,PBS,NA,4,3.843045
4,2,M_2,HMOX1,PBS,PBS,NA,4,3.415457
5,3,M_3,HMOX1,PBS,PBS,NA,4,3.234524
6,4,F_4,HMOX1,PBS,PBS,NA,4,3.294681


# Research Question: Are there statistically significant differences in each biomarker between burn conditions?

Testing for statistical differences by comparing an individual biomarker's value (ie. concentration or ddCT) between burn conditions (ie. control, smoldering, and flaming) within each condensate (ie. plastic and cardboard), dose (only 1 and 25) and time point (only 24).

A Friedman test is a non-paramteric alternative to an ANOVA. A Nemenyi post-hoc will be subsequently run. [ref](https://www.rdocumentation.org/packages/PMCMR/versions/4.3/topics/posthoc.friedman.nemenyi.test) [ref](https://www.statology.org/friedman-test-r/)


# Friedman's Test

In [3]:
# filtering for dose (cytokine conc)
cytokine_df = cytokine_df %>%
    filter(Condensate_Conc %in% c(1, 25, NA))

In [4]:
friedmans_burn_condition = function(df, concentration, value, biomarker, biomarker_name){
    # """
    # Running Friedman's tests comparing burn conditions after filtering for biomarkers, condensate, and
    # concentration. 
    # Ultimately using this test to compare biomarker expression (control, smoldering, flaming).

    # :param: dataframe, concentration variable, value, biomarker name, biomarker variable name
    # :output: a dataframe containing the biomarker, variable name, condensate, concentration,
    # comparison, stat, p value, p adj

    # """
    
    # variables that will be iterated through
    unique_biomarker = unique(df[[biomarker_name]])
    unique_condensate = c("C", "P")
    unique_conc = na.omit(unique(df[[concentration]]))
    
    friedmans_df = data.frame()
    nemenyis_df = data.frame()
    # iterating through each concentration
    for(i in 1:length(unique_biomarker)){
        for(j in 1:length(unique_condensate)){
            for(k in 1:length(unique_conc)){
                  
                # filtering for each biomarker, conc
                filtered_df = df %>%
                    filter(eval(rlang::parse_expr(biomarker_name)) == unique_biomarker[i], Condensate == "PBS" 
                           | Condensate == unique_condensate[j] &
                           eval(rlang::parse_expr(concentration)) == unique_conc[k])


                # friedman's
                friedmans_test = friedman.test(filtered_df[[value]], filtered_df$Burn_Condition, 
                                               filtered_df$Subject_ID)
                friedmans_stat_pvalue = c(friedmans_test$statistic, friedmans_test$p.value)


                # Nemenyi's post hoc
                nemenyis_test = frdAllPairsNemenyiTest(filtered_df[[value]], filtered_df$Burn_Condition, 
                                                       filtered_df$Subject_ID)
                nemenyis_stats = c(nemenyis_test$statistic[1], nemenyis_test$statistic[2], 
                                   nemenyis_test$statistic[4])
                # calculating FC to get directionality
                control_df = filtered_df %>%
                    filter(Condensate == "PBS")
                smoldering_df = filtered_df %>%
                    filter(Burn_Condition == "S")
                flaming_df = filtered_df %>%
                    filter(Burn_Condition == "F")
                
                FC = c(log2(mean(flaming_df[[value]])/mean(control_df[[value]])), # PBS vs. F
                       log2(mean(flaming_df[[value]])/mean(smoldering_df[[value]])), # S vs. F
                       log2(mean(smoldering_df[[value]])/mean(control_df[[value]]))) # PBS vs. S
                
                nemenyis_pvalues = c(nemenyis_test$p.value[1], nemenyis_test$p.value[2], 
                                    nemenyis_test$p.value[4])
                nemenyis_comparison = c(paste0(colnames(nemenyis_test$statistic)[1], " vs. ", 
                                                rownames(nemenyis_test$statistic)[1]),
                                         paste0(colnames(nemenyis_test$statistic)[1], " vs. ", 
                                                rownames(nemenyis_test$statistic)[2]),
                                         paste0(colnames(nemenyis_test$statistic)[2], " vs. ", 
                                                rownames(nemenyis_test$statistic)[2]))


                # contains biomarker, biomarker name, condensate, conc, stat, and p value
                friedmans_vector = cbind(biomarker, unique_biomarker[i], unique_condensate[j], unique_conc[k], 
                                         friedmans_test$statistic, friedmans_test$p.value)
                nemenyis_matrix = cbind(biomarker, unique_biomarker[i], unique_condensate[j], unique_conc[k], 
                                        nemenyis_comparison, nemenyis_stats, FC, nemenyis_pvalues)
                friedmans_df = rbind(friedmans_df, friedmans_vector)
                nemenyis_df = rbind(nemenyis_df, nemenyis_matrix)
                
            }
        }
    }
    
    # adding col names
    colnames(friedmans_df) = c("Biomarker", "Variable Name", "Condensate", "Concentration", 
                               "Statistic", "P Value")
    colnames(nemenyis_df) = c("Biomarker", "Variable Name", "Condensate", "Concentration",  "Comparison", 
                              "Statistic", "FC", "P Value")
    
    # calculating padj values
    friedmans_df = friedmans_df %>%
        group_by(Condensate, Concentration) %>%
        mutate(`P Adj` = p.adjust(as.numeric(as.character(`P Value`)), method = "fdr"))
    nemenyis_df = nemenyis_df %>%
        group_by(Condensate, Concentration) %>%
        mutate(`P Adj` = p.adjust(as.numeric(as.character(`P Value`)), method = "fdr"))

    
    return(list(friedmans_df, nemenyis_df))
}

# calling fn
cytokine_friedmans = friedmans_burn_condition(cytokine_df, "Condensate_Conc", "Norm_Cytokine_Conc", 
                                              "Cytokine", "Cytokine")

Warning message in friedmans_burn_condition(cytokine_df, "Condensate_Conc", "Norm_Cytokine_Conc", :
“NaNs produced”
Warning message in friedmans_burn_condition(cytokine_df, "Condensate_Conc", "Norm_Cytokine_Conc", :
“NaNs produced”
Warning message in friedmans_burn_condition(cytokine_df, "Condensate_Conc", "Norm_Cytokine_Conc", :
“NaNs produced”
Warning message in friedmans_burn_condition(cytokine_df, "Condensate_Conc", "Norm_Cytokine_Conc", :
“NaNs produced”
Warning message in friedmans_burn_condition(cytokine_df, "Condensate_Conc", "Norm_Cytokine_Conc", :
“NaNs produced”
Warning message in friedmans_burn_condition(cytokine_df, "Condensate_Conc", "Norm_Cytokine_Conc", :
“NaNs produced”
Warning message in friedmans_burn_condition(cytokine_df, "Condensate_Conc", "Norm_Cytokine_Conc", :
“NaNs produced”
Warning message in friedmans_burn_condition(cytokine_df, "Condensate_Conc", "Norm_Cytokine_Conc", :
“NaNs produced”
Warning message in friedmans_burn_condition(cytokine_df, "Condensate_Con

In [5]:
# first filtering for dose (1 and 25), time point = 24
split_mRNA_df = mRNA_df %>%
    filter(Concentration %in% c(1,25, NA), Time_Point == 24) %>%
    group_by(Concentration) %>%
    group_split()

conc1_mRNA_df = split_mRNA_df[[1]]
conc25_mRNA_df = split_mRNA_df[[2]]
concc_mRNA_df = split_mRNA_df[[3]]


# control samples have a concentration of NA, so they're all in a separate df
# adding them back into the other dataframes
conc1_mRNA_df = unique(rbind(conc1_mRNA_df, concc_mRNA_df))
conc25_mRNA_df = unique(rbind(conc25_mRNA_df, concc_mRNA_df))

head(conc1_mRNA_df)

Subject_No,Subject_ID,mRNA,Condensate,Burn_Condition,Concentration,Time_Point,Norm_ddCT
<dbl>,<chr>,<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>
4,F_4,HMOX1,P,F,1,24,3.48011747
6,M_6,HMOX1,P,S,1,24,-0.06824032
1,F_1,HMOX1,C,F,1,24,4.63241245
1,F_1,HMOX1,P,F,1,24,1.49163928
1,F_1,HMOX1,P,S,1,24,2.47454559
2,M_2,HMOX1,C,F,1,24,2.43869489


In [6]:
friedmans_burn_condition_v2 = function(df, value, biomarker){
    # """
    # Running Friedman's tests comparing burn conditions after filtering for biomarkers and condensate. 
    # Ultimately using this test to compare biomarker expression (control, smoldering, flaming).

    # :param: dataframe, value, biomarker variable name
    # :output: a dataframe containing the biomarker, variable name, condensate, concentration, time point, 
    # comparison, stat, p value, p adj

    # """
    
    # variables that will be iterated through
    unique_biomarker = unique(df[[biomarker]])
    unique_condensate = c("C", "P")
    
    friedmans_df = data.frame()
    nemenyis_df = data.frame()
    # iterating through each concentration
    for(i in 1:length(unique_biomarker)){
        for(j in 1:length(unique_condensate)){
                  
            # filtering for each biomarker, conc
            filtered_df = df %>%
                filter(eval(rlang::parse_expr(biomarker)) == unique_biomarker[i], 
                       Condensate != unique_condensate[j])


            # friedman's
            friedmans_test = friedman.test(filtered_df[[value]], filtered_df$Burn_Condition, 
                                           filtered_df$Subject_ID)
            friedmans_stat_pvalue = c(friedmans_test$statistic, friedmans_test$p.value)


            # Nemenyi's post hoc
            nemenyis_test = frdAllPairsNemenyiTest(filtered_df[[value]], filtered_df$Burn_Condition, 
                                                   filtered_df$Subject_ID)
            nemenyis_stats = c(nemenyis_test$statistic[1], nemenyis_test$statistic[2], 
                               nemenyis_test$statistic[4])
           # calculating FC to get directionality
            control_df = filtered_df %>%
                filter(Condensate == "PBS")
            smoldering_df = filtered_df %>%
                filter(Burn_Condition == "S")
            flaming_df = filtered_df %>%
                filter(Burn_Condition == "F")
                
            FC = c(log2(mean(flaming_df[[value]])/mean(control_df[[value]])), # PBS vs. F
                   log2(mean(flaming_df[[value]])/mean(smoldering_df[[value]])), # S vs. F
                   log2(mean(smoldering_df[[value]])/mean(control_df[[value]]))) # PBS vs. S
            
            nemenyis_pvalues = c(nemenyis_test$p.value[1], nemenyis_test$p.value[2], 
                                nemenyis_test$p.value[4])
            nemenyis_comparison = c(paste0(colnames(nemenyis_test$statistic)[1], " vs. ", 
                                            rownames(nemenyis_test$statistic)[1]),
                                     paste0(colnames(nemenyis_test$statistic)[1], " vs. ", 
                                            rownames(nemenyis_test$statistic)[2]),
                                     paste0(colnames(nemenyis_test$statistic)[2], " vs. ", 
                                            rownames(nemenyis_test$statistic)[2]))


            # contains biomarker, biomarker name, conc, time point, condensate, stat, and p value
            friedmans_vector = cbind(biomarker, unique_biomarker[i], unique_condensate[j], 
                                     unique(filtered_df$Concentration)[1], friedmans_test$statistic, 
                                     friedmans_test$p.value)
            nemenyis_matrix = cbind(biomarker, unique_biomarker[i], unique_condensate[j],
                                    unique(filtered_df$Concentration)[1], nemenyis_comparison, 
                                    nemenyis_stats, FC, nemenyis_pvalues)
            friedmans_df = rbind(friedmans_df, friedmans_vector)
            nemenyis_df = rbind(nemenyis_df, nemenyis_matrix)

        }
    }
    
    # adding col names
    colnames(friedmans_df) = c("Biomarker", "Variable Name", "Condensate", "Concentration", 
                               "Statistic", "P Value")
    colnames(nemenyis_df) = c("Biomarker", "Variable Name", "Condensate", "Concentration", "Comparison", 
                              "Statistic", "FC", "P Value")
    
    # calculating padj values
    friedmans_df = friedmans_df %>%
        group_by(Condensate) %>%
        mutate(`P Adj` = p.adjust(as.numeric(as.character(`P Value`)), method = "fdr"))
    nemenyis_df = nemenyis_df %>%
        group_by(Condensate) %>%
        mutate(`P Adj` = p.adjust(as.numeric(as.character(`P Value`)), method = "fdr"))

    
    return(list(friedmans_df, nemenyis_df))
}

# calling fn

conc1_friedmans_mRNA_v2 = friedmans_burn_condition_v2(conc1_mRNA_df, "Norm_ddCT", "mRNA")
conc25_friedmans_mRNA_v2 = friedmans_burn_condition_v2(conc25_mRNA_df, "Norm_ddCT", "mRNA")

Warning message in friedmans_burn_condition_v2(conc1_mRNA_df, "Norm_ddCT", "mRNA"):
“NaNs produced”
Warning message in friedmans_burn_condition_v2(conc1_mRNA_df, "Norm_ddCT", "mRNA"):
“NaNs produced”
Warning message in friedmans_burn_condition_v2(conc1_mRNA_df, "Norm_ddCT", "mRNA"):
“NaNs produced”
Warning message in friedmans_burn_condition_v2(conc1_mRNA_df, "Norm_ddCT", "mRNA"):
“NaNs produced”
Warning message in friedmans_burn_condition_v2(conc1_mRNA_df, "Norm_ddCT", "mRNA"):
“NaNs produced”
Warning message in friedmans_burn_condition_v2(conc1_mRNA_df, "Norm_ddCT", "mRNA"):
“NaNs produced”
Warning message in friedmans_burn_condition_v2(conc1_mRNA_df, "Norm_ddCT", "mRNA"):
“NaNs produced”
Warning message in friedmans_burn_condition_v2(conc1_mRNA_df, "Norm_ddCT", "mRNA"):
“NaNs produced”
Warning message in friedmans_burn_condition_v2(conc1_mRNA_df, "Norm_ddCT", "mRNA"):
“NaNs produced”
Warning message in friedmans_burn_condition_v2(conc1_mRNA_df, "Norm_ddCT", "mRNA"):
“NaNs produced”


Warning message in friedmans_burn_condition_v2(conc25_mRNA_df, "Norm_ddCT", "mRNA"):
“NaNs produced”
Warning message in friedmans_burn_condition_v2(conc25_mRNA_df, "Norm_ddCT", "mRNA"):
“NaNs produced”
Warning message in friedmans_burn_condition_v2(conc25_mRNA_df, "Norm_ddCT", "mRNA"):
“NaNs produced”
Warning message in friedmans_burn_condition_v2(conc25_mRNA_df, "Norm_ddCT", "mRNA"):
“NaNs produced”
Warning message in friedmans_burn_condition_v2(conc25_mRNA_df, "Norm_ddCT", "mRNA"):
“NaNs produced”
Warning message in friedmans_burn_condition_v2(conc25_mRNA_df, "Norm_ddCT", "mRNA"):
“NaNs produced”


In [7]:
# creating 1 df
friedmans_df = rbind(cytokine_friedmans[[1]], conc1_friedmans_mRNA_v2[[1]],  conc25_friedmans_mRNA_v2[[1]])
nemenyis_df = rbind(cytokine_friedmans[[2]], conc1_friedmans_mRNA_v2[[2]], conc25_friedmans_mRNA_v2[[2]])

head(friedmans_df)
head(nemenyis_df)

Biomarker,Variable Name,Condensate,Concentration,Statistic,P Value,P Adj
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
Cytokine,Eotaxin3,C,1,2.33333333333333,0.311403223914598,0.3781325
Cytokine,Eotaxin3,C,25,0.333333333333333,0.846481724890614,0.8993868
Cytokine,Eotaxin3,P,1,1.33333333333333,0.513417119032592,0.6234351
Cytokine,Eotaxin3,P,25,0.333333333333333,0.846481724890614,0.8464817
Cytokine,GMCSF,C,1,4.33333333333333,0.114558843992688,0.2300700
Cytokine,GMCSF,C,25,5.33333333333333,0.0694834512228015,0.3895001


Biomarker,Variable Name,Condensate,Concentration,Comparison,Statistic,FC,P Value,P Adj
<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<dbl>
Cytokine,Eotaxin3,C,1,F vs. PBS,0.408248290463863,0.297357398716025,0.955103094750503,1.0000000
Cytokine,Eotaxin3,C,1,F vs. S,1.63299316185545,-0.109199269589443,0.480432575468159,0.9074838
Cytokine,Eotaxin3,C,1,PBS vs. S,2.04124145231932,0.406556668305468,0.318645081847549,0.6771208
Cytokine,Eotaxin3,C,25,F vs. PBS,0.816496580927726,-0.276222424705951,0.832229218116194,1.0000000
Cytokine,Eotaxin3,C,25,F vs. S,0.408248290463863,-0.250733772503139,0.955103094750503,1.0000000
Cytokine,Eotaxin3,C,25,PBS vs. S,0.408248290463863,-0.0254886522028118,0.955103094750503,1.0000000


In [8]:
# exporting
write.xlsx(friedmans_df, paste0(Output,"/", "Friedmans_Burn_Condition_Results_", cur_date, ".xlsx"), 
           rowNames = FALSE)
write.xlsx(nemenyis_df, paste0(Output,"/", "Nemenyis_PostHoc_Burn_Condition_Results_", cur_date, ".xlsx"), 
           rowNames = FALSE)